In [33]:
pip install pymorphy3

In [34]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import math
import pymorphy3

# Скачиваем необходимые ресурсы NLTK

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

morph = pymorphy3.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [35]:
stop_words = set(stopwords.words('russian'))



In [36]:
texts = [
    "Летучие мыши увидели кошек с самыми яркими полосками, висящих вниз головой у них за ноги",
    "Кошка сидит с летучими мышами на полосатом коврике под множеством гусей"
]

In [37]:
def preprocess(text):
    stop_words = set(stopwords.words('russian'))
    tokens = nltk.word_tokenize(text.lower())
    return [morph.parse(t)[0].normal_form for t in tokens if t.isalpha() and t not in stop_words]

def bag_of_words_from_processed(processed_texts):
    all_words = []
    for text_tokens in processed_texts:
        all_words.extend(text_tokens)  # Добавляем токены каждого текста в общий список
    return dict(Counter(all_words))



def tf(text):
    term_count = Counter(text)
    total_terms = len(text)
    return {word: count / total_terms for word, count in term_count.items()}

def idf(texts):
    num_docs = len(texts)
    idf_values = {}  # Переименовал idf в idf_values, чтобы не путать с именем функции
    all_words = set(word for text in texts for word in text)

    for word in all_words:
        containing_docs = sum(1 for text in texts if word in text)
        idf_values[word] = math.log(num_docs / (1 + containing_docs)) + 1

    return idf_values

def tf_idf(texts):
    idf_values = idf(texts)  # Вызываем idf с правильным аргументом
    tf_idf_values = []

    for text in texts:
        tf_values = tf(text) # Вызываем tf c аргументом text
        tf_idf = {word: tf_values[word] * idf_values[word] for word in tf_values}
        tf_idf_values.append(tf_idf)

    return tf_idf_values

In [38]:
processed_texts = [preprocess(text) for text in texts]
print(processed_texts)

[['летучий', 'мышь', 'увидеть', 'кошка', 'самый', 'яркий', 'полоска', 'висеть', 'вниз', 'голова', 'нога'], ['кошка', 'сидеть', 'летучий', 'мышь', 'полосатый', 'коврик', 'множество', 'гусь']]


In [39]:
bow = bag_of_words_from_processed(processed_texts)
print("Bag of Words:", bow)

Bag of Words: {'летучий': 2, 'мышь': 2, 'увидеть': 1, 'кошка': 2, 'самый': 1, 'яркий': 1, 'полоска': 1, 'висеть': 1, 'вниз': 1, 'голова': 1, 'нога': 1, 'сидеть': 1, 'полосатый': 1, 'коврик': 1, 'множество': 1, 'гусь': 1}


In [40]:
result_td_idf = tf_idf(processed_texts)
print(result_td_idf)

[{'летучий': 0.05404862653562142, 'мышь': 0.05404862653562142, 'увидеть': 0.09090909090909091, 'кошка': 0.05404862653562142, 'самый': 0.09090909090909091, 'яркий': 0.09090909090909091, 'полоска': 0.09090909090909091, 'висеть': 0.09090909090909091, 'вниз': 0.09090909090909091, 'голова': 0.09090909090909091, 'нога': 0.09090909090909091}, {'кошка': 0.07431686148647945, 'сидеть': 0.125, 'летучий': 0.07431686148647945, 'мышь': 0.07431686148647945, 'полосатый': 0.125, 'коврик': 0.125, 'множество': 0.125, 'гусь': 0.125}]
